In [132]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [134]:
price_history = yf.Ticker('TSLA').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   actions=False)

In [135]:
price_history

,Open,High,Low,Close,Volume
Date,,,,,
2021-03-01 00:00:00-05:00,230.036667,239.666672,228.350006,239.476669,81408600
2021-03-02 00:00:00-05:00,239.426666,240.369995,228.333328,228.813339,71196600
2021-03-03 00:00:00-05:00,229.330002,233.566666,217.236664,217.733337,90624000
2021-03-04 00:00:00-05:00,218.600006,222.816666,200.000000,207.146667,197758500
2021-03-05 00:00:00-05:00,208.686661,209.279999,179.830002,199.316666,268189500
...,...,...,...,...,...
2023-02-21 00:00:00-05:00,204.990005,209.710007,197.220001,197.369995,180018600
2023-02-22 00:00:00-05:00,197.929993,201.990005,191.779999,200.860001,191828500
2023-02-23 00:00:00-05:00,203.910004,205.139999,196.330002,202.070007,146360000


In [136]:
def find_volatility(ticker):
  data = yf.Ticker(ticker).history(period='3mo', interval='1d', actions=False)
  data['Log returns'] = np.log(data['Close']/data['Close'].shift())
  volatility = data['Log returns'].std()*252**.5
  return volatility

In [137]:
def find_beta_against_spy(ticker):
  data = yf.Ticker(ticker).history(period='12mo', interval='1d', actions=False)
  data['Log returns'] = np.log(data['Close']/data['Close'].shift())
  data_spy = yf.Ticker('SPY').history(period='12mo', interval='1d', actions=False)
  data_spy['Log returns'] = np.log(data_spy['Close']/data_spy['Close'].shift())
  cov = data['Log returns'].cov(data_spy['Log returns'])
  var = data_spy['Log returns'].var()
  beta_spy = cov/var
  return beta_spy

In [138]:
def find_beta_against_iwm(ticker):
  data = yf.Ticker(ticker).history(period='12mo', interval='1d', actions=False)
  data['Log returns'] = np.log(data['Close']/data['Close'].shift())
  data_iwm = yf.Ticker('iwm').history(period='12mo', interval='1d', actions=False)
  data_iwm['Log returns'] = np.log(data_iwm['Close']/data_iwm['Close'].shift())
  cov = data['Log returns'].cov(data_iwm['Log returns'])
  var = data_iwm['Log returns'].var()
  beta_iwm = cov/var
  return beta_iwm

In [139]:
def find_beta_against_dia(ticker):
  data = yf.Ticker(ticker).history(period='12mo', interval='1d', actions=False)
  data['Log returns'] = np.log(data['Close']/data['Close'].shift())
  data_dia = yf.Ticker('dia').history(period='12mo', interval='1d', actions=False)
  data_dia['Log returns'] = np.log(data_dia['Close']/data_dia['Close'].shift())
  cov = data['Log returns'].cov(data_dia['Log returns'])
  var = data_dia['Log returns'].var()
  beta_dia = cov/var
  return beta_dia

In [140]:
def find_average_weekly_drawdown(ticker):
  data = yf.Ticker(ticker).history(period='12mo', interval='1d', actions=False)
  avg_weekly_drawdown = (data['Close'].min() - data['Close'].max())/data['Close'].max()
  return avg_weekly_drawdown

In [141]:
def find_maximum_weekly_drawdown(ticker):
  data = yf.Ticker(ticker).history(period='12mo', interval='1d', actions=False)
  avg_maximum_drawdown = (data['Close'].min() - data['Close'].max())/data['Close'].max()
  return avg_maximum_drawdown

In [142]:
def find_return(ticker):
  data = yf.Ticker(ticker).history(period='10y', interval='3mo', actions=False)
  ret = ((data['Close'].iloc[-1] - data['Close'].iloc[0])/data['Close'].iloc[0])
  return ret

In [143]:
def find_annualized_return(ticker):
  data = yf.Ticker(ticker).history(period='10y', interval='3mo', actions=False)
  ret = ((data['Close'].iloc[-1] - data['Close'].iloc[0])/data['Close'].iloc[0])
  annualized_ret = ((1+ret)**(1/10)) - 1
  return annualized_ret

In [144]:
tickers = ['TSLA', 'AAPL', 'MSFT', 'AMZN', 'UNH', 'GOOGL', 'NVDA']

In [145]:
table1 = pd.DataFrame(columns=['Ticker', 'Portfolio Weight (equally weighted)', 'Annualized Volatility (using trailing 3-months)', 'Beta against SPY (using trailing 12-months)', 'Beta against IWM (using trailing 12-months)', 'Beta against DIA (using trailing 12-months)', 'Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High', 'Maximum Weekly Drawdown (52-week Low minus 52-week High) / 52-week High', 'Total Return (using trailing 10-years)', 'Annualized Total Return (using trailing 10-years)'])

In [146]:
table1['Ticker'] = tickers
table1['Portfolio Weight (equally weighted)'] = [1.0 for i in range(len(tickers))]
table1['Annualized Volatility (using trailing 3-months)'] = list(map(find_volatility, tickers))
table1['Beta against SPY (using trailing 12-months)'] = list(map(find_beta_against_spy, tickers))
table1['Beta against IWM (using trailing 12-months)'] = list(map(find_beta_against_iwm, tickers))
table1['Beta against DIA (using trailing 12-months)'] = list(map(find_beta_against_dia, tickers))
table1['Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High'] = list(map(find_average_weekly_drawdown, tickers))
table1['Maximum Weekly Drawdown (52-week Low minus 52-week High) / 52-week High'] = list(map(find_maximum_weekly_drawdown, tickers))
table1['Total Return (using trailing 10-years)'] = list(map(find_return, tickers))
table1['Annualized Total Return (using trailing 10-years)'] = list(map(find_annualized_return, tickers))

In [147]:
table1.head(len(tickers))

,Ticker,Portfolio Weight (equally weighted),Annualized Volatility (using trailing 3-months),Beta against SPY (using trailing 12-months),Beta against IWM (using trailing 12-months),Beta against DIA (using trailing 12-months),Average Weekly Drawdown (52-week Low minus 52-week High) / 52-week High,Maximum Weekly Drawdown (52-week Low minus 52-week High) / 52-week High,Total Return (using trailing 10-years),Annualized Total Return (using trailing 10-years)
0,TSLA,1.0,0.796812,1.739676,1.515709,1.706969,-0.716880,-0.716880,56.685688,0.500054
1,AAPL,1.0,0.294755,1.303787,1.037119,1.474275,-0.298248,-0.298248,9.893495,0.269746
2,MSFT,1.0,0.325976,1.295466,0.998773,1.466260,-0.317698,-0.317698,8.188563,0.248317
3,AMZN,1.0,0.417194,1.657138,1.361745,1.776833,-0.516759,-0.516759,6.388204,0.221389
4,UNH,1.0,0.228033,0.537093,0.371707,0.768203,-0.188245,-0.188245,8.405080,0.251227
5,GOOGL,1.0,0.400636,1.364588,1.096222,1.476445,-0.416457,-0.416457,3.355246,0.158514
6,NVDA,1.0,0.619781,2.170536,1.861854,2.337318,-0.608015,-0.608015,72.908447,0.537692


In [148]:
etfs = ['EWD', 'SLX', 'CQQQ']

In [149]:
df = yf.Ticker(tickers[0]).history(period='10y', interval='1d', actions=False)['Close']
for t in tickers[1:]:
  df = df + yf.Ticker(t).history(period='10y', interval='1d', actions=False)['Close']

In [150]:
df.head()

Date
2013-02-28 00:00:00-05:00    121.030101
2013-03-01 00:00:00-05:00    121.075366
2013-03-04 00:00:00-05:00    121.550338
2013-03-05 00:00:00-05:00    122.868421
2013-03-06 00:00:00-05:00    122.374424
Name: Close, dtype: float64

In [151]:
df.tail()

Date
2023-02-21 00:00:00-05:00    1482.749992
2023-02-22 00:00:00-05:00    1485.150009
2023-02-23 00:00:00-05:00    1521.280006
2023-02-24 00:00:00-05:00    1492.629997
2023-02-27 00:00:00-05:00    1507.670013
Name: Close, dtype: float64

In [152]:
df/=len(tickers)

In [153]:
df.tail()

Date
2023-02-21 00:00:00-05:00    211.821427
2023-02-22 00:00:00-05:00    212.164287
2023-02-23 00:00:00-05:00    217.325715
2023-02-24 00:00:00-05:00    213.232857
2023-02-27 00:00:00-05:00    215.381430
Name: Close, dtype: float64

In [197]:
def find_etf_correlation(ticker):
  data = yf.Ticker(ticker).history(period='10y', interval='1d', actions=False)['Close']
  corr = data.corr(df)
  return corr

In [198]:
def find_etf_covariance(ticker):
  data = yf.Ticker(ticker).history(period='10y', interval='1d', actions=False)['Close']
  cov = data.cov(df)
  return cov

In [199]:
def find_etf_tracking_errors(ticker):
  data = yf.Ticker(ticker).history(period='10y', interval='1d', actions=False)['Close']
  tracking_error = np.std((df.values - data.values) / df.values)
  return tracking_error

In [200]:
def find_etf_sharpe_ratio(ticker):
  data = yf.Ticker(ticker).history(period='10y', interval='1d', actions=False)['Close'].pct_change().dropna()
  data = data.values
  risk_free_Rate = 0.0
  mean_daily_return = sum(data)/len(data)
  std = np.std(data)
  daily_sharpe_ratio = (mean_daily_return - risk_free_Rate) / std
  sharpe_ratio = 252**(1/2) * daily_sharpe_ratio
  return sharpe_ratio

In [201]:
def find_etf_annualized_volatility_spread(ticker):
  data = yf.Ticker(ticker).history(period='10y', interval='1d', actions=False)
  data['Log returns'] = np.log(data['Close']/data['Close'].shift())
  etf_volatility = data['Log returns'].std()*252**.5
  portfolio_log_returns = np.log(df/df.shift())
  portfolio_volatility = portfolio_log_returns.std()*252**.5
  return portfolio_volatility - etf_volatility

In [202]:
table2 = pd.DataFrame(columns=['ETF Ticker', 'Correlation against ETF', 'Covariance of Portfolio against ETF', 'Tracking Errors (using trailing 10-years)', 'Sharpe Ratio (using current risk-free rate)', 'Annualized Volatility (252 days) Spread (Portfolio Volatility – ETF Volatility)'])

In [203]:
table2['ETF Ticker'] = etfs
table2['Correlation against ETF'] = list(map(find_etf_correlation, etfs))
table2['Covariance of Portfolio against ETF'] = list(map(find_etf_covariance, etfs))
table2['Tracking Errors (using trailing 10-years)'] = list(map(find_etf_tracking_errors, etfs))
table2['Sharpe Ratio (using current risk-free rate)'] = list(map(find_etf_sharpe_ratio, etfs))
table2['Annualized Volatility (252 days) Spread (Portfolio Volatility – ETF Volatility)'] = list(map(find_etf_annualized_volatility_spread, etfs))

In [204]:
table2.head()

,ETF Ticker,Correlation against ETF,Covariance of Portfolio against ETF,Tracking Errors (using trailing 10-years),Sharpe Ratio (using current risk-free rate),Annualized Volatility (252 days) Spread (Portfolio Volatility – ETF Volatility)
0,EWD,0.874130,415.070819,0.302942,0.320300,0.003271
1,SLX,0.794261,710.282297,0.385798,0.387549,-0.091469
2,CQQQ,0.667982,813.133906,0.340953,0.375248,-0.073745


In [212]:
table3 = pd.DataFrame()
table3['EquallyWeightedPortfolio'] = df
correlation_matrix_tickers = etfs + tickers
for t in correlation_matrix_tickers:
  table3[t] = yf.Ticker(t).history(period='10y', interval='1d', actions=False)['Close']

In [213]:
table3.corr()

,EquallyWeightedPortfolio,EWD,SLX,CQQQ,TSLA,AAPL,MSFT,AMZN,UNH,GOOGL,NVDA
EquallyWeightedPortfolio,1.000000,0.874130,0.794261,0.667982,0.945736,0.991506,0.991801,0.917271,0.966159,0.983111,0.975522
EWD,0.874130,1.000000,0.790014,0.787748,0.858171,0.859056,0.869914,0.845088,0.773046,0.903593,0.896157
SLX,0.794261,0.790014,1.000000,0.458323,0.772197,0.785749,0.757554,0.620575,0.795778,0.786078,0.803811
CQQQ,0.667982,0.787748,0.458323,1.000000,0.576914,0.641645,0.684314,0.830828,0.590329,0.693499,0.647499
TSLA,0.945736,0.858171,0.772197,0.576914,1.000000,0.949035,0.917264,0.807319,0.862377,0.924280,0.938622
AAPL,0.991506,0.859056,0.785749,0.641645,0.949035,1.000000,0.983074,0.892662,0.956960,0.961525,0.960729
MSFT,0.991801,0.869914,0.757554,0.684314,0.917264,0.983074,1.000000,0.936343,0.954339,0.979088,0.962923
AMZN,0.917271,0.845088,0.620575,0.830828,0.807319,0.892662,0.936343,1.000000,0.863858,0.919647,0.878229
UNH,0.966159,0.773046,0.795778,0.590329,0.862377,0.956960,0.954339,0.863858,1.000000,0.935201,0.914189
GOOGL,0.983111,0.903593,0.786078,0.693499,0.924280,0.961525,0.979088,0.919647,0.935201,1.000000,0.970195


In [214]:
table3.head()

,EquallyWeightedPortfolio,EWD,SLX,CQQQ,TSLA,AAPL,MSFT,AMZN,UNH,GOOGL,NVDA
Date,,,,,,,,,,,
2013-02-28 00:00:00-05:00,17.290014,22.332064,31.586666,21.968330,2.322000,13.615319,23.055151,13.2135,45.832729,20.050051,2.941361
2013-03-01 00:00:00-05:00,17.296481,22.532320,31.019159,21.903473,2.310000,13.278174,23.179544,13.2870,45.892742,20.174925,2.952978
2013-03-04 00:00:00-05:00,17.364334,22.787815,30.527773,21.866415,2.372000,12.956760,23.345404,13.6555,45.721252,20.558058,2.941361
2013-03-05 00:00:00-05:00,17.552632,22.960451,30.776926,22.292624,2.443333,13.298837,23.511282,13.7795,45.875595,20.985987,2.973888
2013-03-06 00:00:00-05:00,17.482061,22.815439,31.524376,22.135111,2.512667,13.129802,23.295649,13.6895,45.969925,20.805305,2.971565
